### Desabilitando os warnings

In [150]:
import warnings
warnings.filterwarnings('ignore')

### Importando bibliotecas

In [151]:
import pandas as pd

### Importando conjuntos de dados tratados

In [152]:
df_orders = pd.read_parquet(
    r"..\data\df_orders.parquet")

df_person = pd.read_parquet(
    r"..\data\df_people.parquet")

df_returns = pd.read_parquet(
    r"..\data\df_returns.parquet")

### Análise de Vendas por Região e Responsável

- Insight: Quais regiões estão gerando mais vendas e quais são os responsáveis por elas?

- Dashboard: Um gráfico de barras empilhadas ou um mapa interativo que mostre as vendas por
região, destacando o responsável (usando a tabela "people"). Isso ajuda a identificar a performance
por líder de cada região

In [153]:
df_sales_x_region = df_orders[["Sales","Region"]].groupby(["Region"]).sum()[["Sales"]].reset_index()
df_sales_x_region = df_sales_x_region[["Region", "Sales"]].merge(df_person, on="Region", how="left")
df_sales_x_region = df_sales_x_region.sort_values(by="Sales", ascending=False)

In [154]:
df_sales_x_region["Region"] = df_sales_x_region["Region"].astype("category")
df_sales_x_region["Person"] = df_sales_x_region["Person"].astype("category")
df_sales_x_region["Sales"] = df_sales_x_region["Sales"].apply(pd.to_numeric, downcast = "integer")

In [155]:
df_sales_x_region.head(3)

,Region,Sales,Person
3,Central,2.822303e+06,Anna Andreadi
10,South,1.600907e+06,Chuck Magee
7,North,1.248166e+06,Jack Lebron


In [156]:
df_sales_x_region.to_parquet(r"..\reports\Análise de Vendas por Região e Responsável\df_sales_x_region.parquet")

###  Análise de Devoluções por Mercado

- Insight: Quais mercados têm o maior número de devoluções? Como isso afeta as vendas e o lucro?

- Dashboard: Gráfico de barras mostrando o número de devoluções por mercado (LATAM, EU,
United States). Pode-se cruzar com o impacto nas vendas e no lucro, mostrando quanto de receita
foi perdida por devoluções.

In [157]:
df_devol_merc = (df_orders[["Order ID", "Market", "Sales"]]
                 .merge(df_returns,
                       on=['Order ID',"Market"], 
                       how= "left"))

df_devol_merc["Returned"] = (df_devol_merc["Returned"]
                             .astype(str)
                             .replace("nan", "Não Devolvido"))

df_devol_merc_devolvidos = df_devol_merc.loc[df_devol_merc["Returned"] == "Devolvido"]
df_devol_merc_nao_devolvidos = df_devol_merc.loc[df_devol_merc["Returned"] == "Não Devolvido"]

In [158]:
df_devol_merc = (df_devol_merc[["Market", "Sales"]]
                 .groupby("Market").sum()
                 .sort_values(by="Sales",
                              ascending=False))

df_devol_merc_devolvidos = (df_devol_merc_devolvidos[["Market", "Sales"]]
                            .groupby("Market").sum()
                            .sort_values(by="Sales",
                                         ascending=False))

df_devol_merc_nao_devolvidos = (df_devol_merc_nao_devolvidos[["Market", "Sales"]]
                                .groupby("Market").sum()
                                .sort_values(by="Sales",
                                             ascending=False))

In [159]:
df_devol_merc.to_parquet(r"..\reports\Análise de Devoluções por Mercado\df_devol_merc.parquet")
df_devol_merc_devolvidos.to_parquet(r"..\reports\Análise de Devoluções por Mercado\df_devol_merc_devolvidos.parquet")
df_devol_merc_nao_devolvidos.to_parquet(r"..\reports\Análise de Devoluções por Mercado\df_devol_merc_nao_devolvidos.parquet")

### Análise de Vendas e Devoluções por Produto

- Insight: Quais produtos têm maior número de devoluções? Isso está impactando a lucratividade?

- Dashboard: Uma tabela ou gráfico de barras que combine os dados de vendas e devoluções por
produto. Produtos com alta taxa de devolução podem estar prejudicando o desempenho financeiro.

In [160]:
df_devol_prod = (df_orders[
    ["Order ID", "Market", "Product Name", "Sales"]]
    .merge(df_returns,
           on=['Order ID',"Market"], 
           how= "left"))

df_devol_prod["Returned"] = (df_devol_prod["Returned"]
                             .astype(str)
                             .replace("nan", "Não Devolvido"))

df_devol_prod_devolvidos = (
    df_devol_prod.loc[df_devol_prod["Returned"] == "Devolvido"])

In [161]:
df_devol_prod = (df_devol_prod[["Product Name", "Sales"]]
                 .groupby("Product Name").sum()
                 .sort_values(by="Sales",
                              ascending=False))

df_devol_prod_devolvidos = (df_devol_prod_devolvidos[
    ["Product Name", "Sales"]]
    .groupby("Product Name").sum()
    .sort_values(by="Sales",
                 ascending=False))
df_devol_prod_devolvidos.columns = ["Returns"]

df_devol_prod = (df_devol_prod
                 .merge(df_devol_prod_devolvidos,
                        on="Product Name",
                        how="left"))

df_devol_prod = df_devol_prod.dropna()
# df_devol_prod["Returns"] = df_devol_prod["Returns"].fillna(0)

df_devol_prod["tx.Return"] = (
    df_devol_prod["Returns"]/df_devol_prod["Sales"])
df_devol_prod = (df_devol_prod
                 .sort_values(by="tx.Return",
                              ascending=False))
df_devol_prod["tx.Return"] = df_devol_prod["tx.Return"] * 100


df_devol_prod

,Sales,Returns,tx.Return
Product Name,,,
Acco Glide Clips,19.600000,19.600000,100.000000
Okidata B401 Printer,179.990997,179.990997,100.000000
Avery 500,58.480000,58.480000,100.000000
Canon Color Imageclass Mf8580Cdw Wireless Laser AllInOne Printer Copier Scanner,1007.979004,1007.979004,100.000000
Zebra Gk420T Direct ThermalThermal Transfer Printer,703.710022,703.710022,100.000000
...,...,...,...
Bic Pens EasyErase,1410.719517,9.700000,0.687592
Ibico 3Hole Punch Clear,2615.031311,17.945999,0.686263
Ibico Binder Covers Recycled,1639.963919,11.208000,0.683430


In [162]:
# df_devol_prod_tx_devol = df_devol_prod.copy()
# df_devol_prod_tx_devol.columns = ["Total"]
# df_devol_prod_tx_devol["Returns"] =  df_devol_prod_devolvidos["Sales"]
# df_devol_prod_tx_devol["Returns"] =  df_devol_prod_tx_devol["Returns"].fillna(0)
# df_devol_prod_tx_devol["tx.Return"] = (
#     df_devol_prod_tx_devol["Returns"] / df_devol_prod_tx_devol["Total"])

# df_devol_prod_tx_devol = df_devol_prod_tx_devol[["tx.Return"]]

In [163]:
df_devol_prod.to_parquet(
    r"..\reports\Análise de Vendas e Devoluções por Produto\df_devol_prod.parquet")
# df_devol_prod_devolvidos.to_parquet(
#     r"..\reports\Análise de Vendas e Devoluções por Produto\df_devol_prod_devolvidos.parquet")

# df_devol_prod_tx_devol.to_parquet(
#     r"..\reports\Análise de Vendas e Devoluções por Produto\df_devol_prod_tx_devol.parquet")

###  Análise de Lucratividade por Produto e Devoluções

- Insight: Quais produtos continuam lucrativos mesmo após devoluções?


- Dashboard: Gráfico de barras empilhadas que mostre o lucro antes e depois de considerar as
devoluções por produto.

###  Custo de Envio e Devoluções por País

- Insight: Quais países têm maior custo de envio e também sofrem mais com devoluções? Isso está
corroendo os lucros?

- Dashboard: Gráfico de barras ou mapa com o custo de envio médio por país, destacando as
devoluções.

### Análise de Prioridade de Pedidos e Devoluções

- Insight: Existe uma relação entre a prioridade dos pedidos e as devoluções? Pedidos com alta
prioridade têm mais devoluções?

- Dashboard: Gráfico de rosca ou barras mostrando a distribuição de devoluções por prioridade de
pedido (Critical, Medium).


### Correlação entre Devoluções e Desconto

- Insight: Produtos com desconto estão mais propensos a serem devolvidos?

- Dashboard: Gráfico de dispersão que relacione o percentual de desconto com o número de
devoluções por produto.


### Análise de Performance por Líder de Região

- Insight: Como os líderes de cada região (usando a tabela "people") estão performando em termos
de vendas, devoluções e lucro?

- Dashboard: Um gráfico comparativo de vendas, devoluções e lucro por líder de região.

### Devoluções e Impacto no Lucro

- Insight: Quais mercados ou produtos estão sofrendo mais com devoluções e como isso impacta o
lucro total da empresa?

- Dashboard: Um gráfico de barras empilhadas que mostre as vendas totais e o impacto das
devoluções no lucro por mercado.

### Margem de Lucro por Produto Considerando Devoluções

- Insight: Quais produtos têm margens de lucro mais altas, mesmo considerando as devoluções?

- Dashboard: Gráfico de barras horizontais com a margem de lucro (%) por produto, incluindo o
impacto das devoluções.